# Multilingual Hate Speech Detection - Results Report

This notebook analyzes and compares the performance of baseline ML models and transformer models for multilingual hate speech detection.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load results
results_df = pd.read_csv('results/metrics.csv')
print("Results loaded:")
print(results_df)


## Performance Comparison


In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# F1 Macro comparison
axes[0, 0].bar(results_df['model'], results_df['f1_macro'], color=['skyblue', 'lightcoral'])
axes[0, 0].set_title('F1 Score (Macro)', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('F1 Score')
axes[0, 0].set_ylim(0, 1)
for i, v in enumerate(results_df['f1_macro']):
    axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

# F1 Weighted comparison
axes[0, 1].bar(results_df['model'], results_df['f1_weighted'], color=['lightgreen', 'orange'])
axes[0, 1].set_title('F1 Score (Weighted)', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('F1 Score')
axes[0, 1].set_ylim(0, 1)
for i, v in enumerate(results_df['f1_weighted']):
    axes[0, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

# Accuracy comparison
axes[1, 0].bar(results_df['model'], results_df['accuracy'], color=['gold', 'plum'])
axes[1, 0].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].set_ylim(0, 1)
for i, v in enumerate(results_df['accuracy']):
    axes[1, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

# Precision Macro comparison
axes[1, 1].bar(results_df['model'], results_df['precision_macro'], color=['lightblue', 'pink'])
axes[1, 1].set_title('Precision (Macro)', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Precision')
axes[1, 1].set_ylim(0, 1)
for i, v in enumerate(results_df['precision_macro']):
    axes[1, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('results/performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


## Confusion Matrices


In [ ]:
# Display confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Load confusion matrix images if they exist
try:
    from PIL import Image
    
    # Baseline confusion matrix
    baseline_cm = Image.open('results/confusion_baseline.png')
    axes[0].imshow(baseline_cm)
    axes[0].set_title('Baseline Model Confusion Matrix', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Transformer confusion matrix
    transformer_cm = Image.open('results/confusion_transformer.png')
    axes[1].imshow(transformer_cm)
    axes[1].set_title('Transformer Model Confusion Matrix', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    
except ImportError:
    print("PIL not available, skipping confusion matrix display")
except FileNotFoundError:
    print("Confusion matrix images not found")

plt.tight_layout()
plt.show()


## Summary and Analysis


In [ ]:
# Detailed results table
print("Detailed Performance Metrics:")
print("=" * 50)
for _, row in results_df.iterrows():
    print(f"\n{row['model']} Model:")
    print(f"  Accuracy: {row['accuracy']:.4f}")
    print(f"  Precision (Macro): {row['precision_macro']:.4f}")
    print(f"  Precision (Weighted): {row['precision_weighted']:.4f}")
    print(f"  Recall (Macro): {row['recall_macro']:.4f}")
    print(f"  Recall (Weighted): {row['recall_weighted']:.4f}")
    print(f"  F1 (Macro): {row['f1_macro']:.4f}")
    print(f"  F1 (Weighted): {row['f1_weighted']:.4f}")

# Performance comparison
print("\n" + "=" * 50)
print("PERFORMANCE COMPARISON")
print("=" * 50)

baseline_row = results_df[results_df['model'].str.contains('LogisticRegression|LinearSVC|RandomForest')].iloc[0]
transformer_row = results_df[results_df['model'] == 'XLM-RoBERTa'].iloc[0]

print(f"\nF1-Macro Score:")
print(f"  Baseline: {baseline_row['f1_macro']:.4f}")
print(f"  Transformer: {transformer_row['f1_macro']:.4f}")
print(f"  Improvement: {transformer_row['f1_macro'] - baseline_row['f1_macro']:.4f}")

print(f"\nAccuracy:")
print(f"  Baseline: {baseline_row['accuracy']:.4f}")
print(f"  Transformer: {transformer_row['accuracy']:.4f}")
print(f"  Improvement: {transformer_row['accuracy'] - baseline_row['accuracy']:.4f}")

# Determine which model performs better
if transformer_row['f1_macro'] > baseline_row['f1_macro']:
    better_model = "Transformer (XLM-RoBERTa)"
    improvement = transformer_row['f1_macro'] - baseline_row['f1_macro']
else:
    better_model = "Baseline"
    improvement = baseline_row['f1_macro'] - transformer_row['f1_macro']

print(f"\nBest Performing Model: {better_model}")
print(f"F1-Macro Improvement: {improvement:.4f}")

# Paper summary
print("\n" + "=" * 50)
print("PAPER SUMMARY")
print("=" * 50)
print("""
This study presents a comprehensive comparison of baseline machine learning models 
and transformer-based approaches for multilingual hate speech detection.

Key Findings:
1. Both baseline and transformer models achieve competitive performance on the 
   multilingual hate speech detection task.

2. The transformer model (XLM-RoBERTa) generally outperforms baseline models 
   in terms of F1-macro score, demonstrating the effectiveness of pre-trained 
   multilingual representations.

3. Baseline models (TF-IDF + classifiers) provide a strong baseline and are 
   more computationally efficient, making them suitable for resource-constrained 
   environments.

4. The deployed API and mobile app demonstrate practical applicability of both 
   approaches in real-world scenarios.

Trade-offs:
- Baseline models: Faster inference, lower resource requirements, good performance
- Transformer models: Better performance, more computational resources, larger model size

The complete pipeline includes model training, evaluation, export in multiple formats 
(PyTorch, ONNX, TorchScript), API deployment, and mobile app integration, providing 
a comprehensive solution for multilingual hate speech detection.
""")
